In [14]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    def entropy(self, y):
        histogram = np.bincount(y)
        px = histogram / len(y)
        entropy = -np.sum(px * np.log2(px + 1e-10))  # Adding a small value for numerical stability
        return entropy

    def split(self, x_column, split_thresh):
        left_index = np.argwhere(x_column <= split_thresh).flatten()
        right_index = np.argwhere(x_column > split_thresh).flatten()
        return left_index, right_index
        
    def information_gain(self, y, x_column, split_thresh):
        # parent entropy
        parent_entropy = self.entropy(y)
        
        # create children
        left_index, right_index = self.split(x_column, split_thresh)
        
        if len(left_index) == 0 or len(right_index) == 0:
            return 0
        
        # calculate the weighted average entropy of children
        n = len(y)
        n_left, n_right = len(left_index), len(right_index)
        entropy_left, entropy_right = self.entropy(y[left_index]), self.entropy(y[right_index])
        
        child_entroy = (n_left/n)*entropy_left + (n_right/n)*entropy_right
        
        # calculate IG
        info_gain = parent_entropy - child_entroy
        return info_gain

    def best_split(self, X, y, feat_index):
        best_gain = -1
        split_index, split_thresh = None, None

        for feat in feat_index:
            if feat >= X.shape[1]:
                continue  # Skip if feat index is out of bounds

            x_column = X[:, feat]
            thresholds = np.unique(x_column)

            for threshold in thresholds:
                # calculate the information gain
                gain = self.information_gain(y, x_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_index = feat
                    split_thresh = threshold

        return split_index, split_thresh

                
    def most_common_label(self, y):
        counter = Counter(y)
        if len(counter) == 0:
            return None  # Return a default value or handle accordingly
        most_common = counter.most_common(1)[0][0]
        return most_common

    def grow_tree(self, X, y, depth=0):
        
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        
        #Checking the stopping criteria
        if (depth>=self.max_depth or n_labels == 1 or n_samples< self.min_samples_split):
            value = self.most_common_label(y)
            return Node(value=value)
        
        feat_index = np.random.choice(n_features, self.n_features, replace=False)
        
        #Find the best split
        best_thresh, best_feat = self.best_split(X, y, feat_index)        
        
        #Create Child Nodes
        left_index, right_index = self.split(X[:, int(best_feat)], best_thresh)

        left = self.grow_tree(X[left_index, :], y[left_index], depth+1)
        right = self.grow_tree(X[right_index, :], y[right_index], depth+1)
        
        return Node(best_feat, best_thresh, left, right)
    
    def traverse_tree(self, x, node):
        #Make sure is a leaf node
        if node.is_leaf_node():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self.traverse_tree(x, node.left)
        return self.traverse_tree(x, node.right)
    
    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self.grow_tree(X, y)        
        
    def predict(self, X):
        return np.array([self.traverse_tree(x, self.root) for x in X])


In [15]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
data = datasets.load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 18)

clf = DecisionTree(max_depth = 10)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred)/len(y_true)
    return accuracy

acc = accuracy(y_test, predictions)
print(acc)

IndexError: index 107 is out of bounds for axis 1 with size 30